In [1]:
from typing import ClassVar, Dict
import pandas as pd
from pydantic import BaseModel, Field
from neontology import BaseNode, BaseRelationship, init_neontology, auto_constrain

import config

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "abcd1234"

In [2]:
init_neontology(
    neo4j_uri=NEO4J_URI,
    neo4j_username=NEO4J_USERNAME,
    neo4j_password=NEO4J_PASSWORD
)


c:\Users\saran\anaconda3\envs\py3.7\lib\site-packages\neontology\graphconnection.py:43: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


In [3]:
class PinNode(BaseNode):
    __primarylabel__: ClassVar[str] = "ID"
    __primaryproperty__: ClassVar[str] = "pin_id"
    
    pin_id: str
    pin_type: str
    kostenstelle: int
    arbeitsfolge: str
    bearbeitungsschritt: str
    datum: str
    zeit: str
    ergebnis: str
    ausfuerungszeit: float
    anzeigezeit: int


class FeatureNode(BaseNode):
    __primarylabel__: ClassVar[str] = "Feature"
    __primaryproperty__: ClassVar[str] = "name"
    # pin_id: str
    name: str
    value: float
    upper: float
    lower: float

class LabelNode(BaseNode):
    __primarylabel__: ClassVar[str] = "Label"
    __primaryproperty__: ClassVar[str] = "label"
    label: int

class HasFeatureRelationship(BaseRelationship):
    __relationshiptype__: ClassVar[str] = "HAS_FEATURE"
    
    source: PinNode
    target: FeatureNode


class HasLabelRelationship(BaseRelationship):
    __relationshiptype__: ClassVar[str] = "HAS_LABEL"
    
    source: PinNode
    target: LabelNode

In [4]:
df = pd.read_csv(r'D:\MyDocs\DE\Projects\Project_Clustering\data\Combined\20220202_15_7512_0030_Drahtprüfung.csv', delimiter=';')
df = df.iloc[:10, :]
df['label'] = 1
for _, row in df.iterrows():
    pin_node = PinNode(pin_id=row['Pin_ID'], pin_type=row['Pin_Type'], kostenstelle=row['Kostenstelle'], arbeitsfolge=row['Arbeitsfolge'], 
                        bearbeitungsschritt=row['Bearbeitungsschritt'], datum=row['Datum'], zeit=row['Zeit'], ergebnis=row['Ergebnis'], 
                        ausfuerungszeit=row['Ausfuehrungszeit'], anzeigezeit=row['Anzeigezeit'])
    pin_node.create()

    feature_nodes = []
    for feature in config.features_with_threshold.keys():
        feature_node = FeatureNode(
            name=feature,
            value=row[config.features_with_threshold[feature][0]],
            upper=row[config.features_with_threshold[feature][1]],
            lower=row[config.features_with_threshold[feature][2]]
        )
        feature_node.create()
        feature_nodes.append(feature_node)


    label_node = LabelNode(label=row['label'])
    label_node.create()

    # Create relationships
    for feature_node in feature_nodes:
        rel = HasFeatureRelationship(source=pin_node, target=feature_node)
        rel.merge()
    
    rel = HasLabelRelationship(source=pin_node, target=label_node)
    rel.merge()
